In [3]:
from langchain_community.document_loaders import Docx2txtLoader, TextLoader, PyPDFLoader
import os
import inspect


print(inspect.getsourcefile(Docx2txtLoader))
print(inspect.getsourcefile(TextLoader))

d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_community\document_loaders\word_document.py
d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_community\document_loaders\text.py


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

print(inspect.getsourcefile(RecursiveCharacterTextSplitter))

d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_text_splitters\character.py
